# Обучение на малом объеме данных. 7 лекция

Transfer learning, Augmentation. One-shot learning. Кластеризация (embedding, one-shot learning)

## Рекуррентные сети: введение

Мы уже знакомы с основными принципами построения архитектур для самых простых нейросетей. Обычные модели состоят из группы слоёв и принимают на вход объект фиксированного размера — например, изображение. Каждый слой применяет к этому объекту какие-либо преобразования, и на выходе мы получаем результат — для классификации это будет метка класса. Но в некоторых областях машинного обучения нам хотелось бы иметь большую гибкость в типах данных, которые могли бы обрабатывать наши модели.

Именно это позволяют делать рекуррентные нейронные сети (Recurrent Neural Network, RNN). Иногда их делят на несколько разновидностей: “one to one”, “one to many”, “many to one” и “many to many”.

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/L08_0.png" width="700">